Note: \
It might be much faster using asynchronous programming and parsel: import asyncio, import aiohttp, from parsel import Selector, lxml was faster too \
Asyncio with Multiprocessing do this for faster return time for later use


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Note: you may need to restart the kernel to use updated packages.


In [50]:
def getClubData(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    all_links = soup.find_all("a")
    links = [l.get("href") for l in all_links]
    links = [l for l in links if l and param in l]
    links = "https://fbref.com" + links[0]
    print(links)
    
url = "https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats"
getAllClubLinks(url)

[]

In [67]:
import concurrent.futures
import time

def getAllClubData(url):
    # runs only once for all clubs, returns team links
    allClubLinks = getAllClubLinks(url)
    names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
    
    allClubData = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit each club processing as a separate task
        club_tasks = [executor.submit(processClubData, link) for link in allClubLinks]
        
        # Wait for all tasks to complete
        concurrent.futures.wait(club_tasks)
        
        # Get results from completed tasks
        for task in club_tasks:
            club_data = task.result()
            if club_data is not None:
                allClubData.append(club_data)
            time.sleep(1000)
    
    result = pd.concat(allClubData, axis=0)
    return result

def getClubData(requestData, param, match, keepList):
    soup = BeautifulSoup(requestData.text)
    all_links = soup.find_all("a")
    links = [l.get("href") for l in all_links]
    links = [l for l in links if l and param in l]
    links = "https://fbref.com" + links[0]
    shooting = requests.get(links)
    shooting = pd.read_html(shooting.text, match=match)[0]
    shooting.columns = shooting.columns.droplevel()
    shooting = shooting[keepList]
    return shooting[:-1]

def processClubData(link):
    club = requests.get(link)
    clubName = link.split("/")[-1].replace("-Stats","").replace("-", " ")
    print(clubName)

    # get Scores (on the same page)
    scores = pd.read_html(club.text, match = "Scores & Fixtures")[0]
    scores = scores[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]

    # get shooting and other stat (figure better ways for these two)
    shooting = getClubData(club, "/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
    misc = getClubData(club, "/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])
    names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
    finalClubData = scores.merge(shooting, how='left').merge(misc, how='left')
    finalClubData.rename(columns=names, inplace=True)
    finalClubData = finalClubData[finalClubData["Competition"] == "Premier League"]
    finalClubData["Team"] = clubName
    print(clubName, finalClubData)
    return finalClubData

url = "https://fbref.com/en/comps/9/Premier-League-Stats"
getAllClubData(url)

Manchester City
Arsenal
Manchester United
Newcastle United
Liverpool
Brighton and Hove Albion
Aston Villa
Tottenham Hotspur
Brentford


KeyboardInterrupt: 

In [54]:
#get links to each club
def getAllClubLinks(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    team_table = soup.select("table.stats_table")[0]
    team_links = team_table.find_all("a")
    team_links = [l.get("href") for l in team_links]
    team_links = [l for l in team_links if "/squads/" in l]
    team_links = [f"https://fbref.com{l}" for l in team_links]
    return team_links

def getClubData(requestData, param, match, keepList):
    soup = BeautifulSoup(requestData.text)
    all_links = soup.find_all("a")
    links = [l.get("href") for l in all_links]
    links = [l for l in links if l and param in l]
    links = "https://fbref.com" + links[0]
    shooting = requests.get(links)
    shooting = pd.read_html(shooting.text, match=match)[0]
    shooting.columns = shooting.columns.droplevel()
    shooting = shooting[keepList]
    return shooting[:-1]

def getAllClubData(url):
    
    #runs only once for all clubs, returns team links
    allClubLinks = getAllClubLinks(url)
    names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
    
    allClubData = []
    for link in allClubLinks:
        club = requests.get(link)
        clubName = link.split("/")[-1].replace("-Stats","").replace("-", " ")
        print(clubName)
        
        #get Scores (on the same page)
        scores = pd.read_html(club.text, match = "Scores & Fixtures")[0]
        scores = scores[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]
        
        #get shooting and other stat (figure better ways for these two)
        shooting = getClubData(club, "/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
        misc = getClubData(club, "/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])
        
        finalClubData = scores.merge(shooting,how ='left').merge(misc, how='left')
        allClubData.append(finalClubData)
        time.sleep(10)
#     keepList = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss', 'Sh', "SoT", 'CrdY', 'CrdR', 'Fls', 'Off']
    finalClubData.rename(columns=names, inplace=True)
    finalClubData = finalClubData[finalClubData["Competition"] == "Premier League"]
    finalClubData["Team"] = clubName
    result = pd.concat(allClubData, axis=0)
    return result

url = "https://fbref.com/en/comps/9/Premier-League-Stats"
getAllClubData(url)

Process SpawnProcess-5:
Process SpawnProcess-6:
Process SpawnProcess-4:
Process SpawnProcess-2:
Process SpawnProcess-1:
Process SpawnProcess-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/vega/mambaforge/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/vega/mambaforge/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vega/mambaforge/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/vega/mambaforge/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/vega/mambaforge/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vega/mambaforge/lib/python3.10/multiprocessing/queues.py", line 122, in get
    re

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [12]:
#get links to each club
def getAllClubLinks(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    team_table = soup.select("table.stats_table")[0]
    team_links = team_table.find_all("a")
    team_links = [l.get("href") for l in team_links]
    team_links = [l for l in team_links if "/squads/" in l]
    team_links = [f"https://fbref.com{l}" for l in team_links]
    return team_links

url = "https://fbref.com/en/comps/9/Premier-League-Stats"
getAllClubLinks(url)

In [24]:
def getClubData(requestData, param, match, keepList):
    soup = BeautifulSoup(requestData.text)
    all_links = soup.find_all("a")
    links = [l.get("href") for l in all_links]
    links = [l for l in links if l and param in l]
    links = "https://fbref.com" + links[0]
    shooting = requests.get(links)
    shooting = pd.read_html(shooting.text, match=match)[0]
    shooting.columns = shooting.columns.droplevel()
#     shooting = shooting[keepList]
    return shooting[:-1]

In [15]:
def getAllClubData(url):
    
    #runs only once for all clubs, returns team links
    allClubLinks = getAllClubLinks(url)
    names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
    
    allClubData = []
    for link in allClubLinks:
        club = requests.get(link)
        clubName = link.split("/")[-1].replace("-Stats","").replace("-", " ")
        print(clubName)
        
        #get Scores
        scores = pd.read_html(club.text, match = "Scores & Fixtures")[0]
        scores = scores[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]
        
        #get shooting and other stat (figure better ways for these two)
        shooting = getClubData(club, "/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
        misc = getClubData(club, "/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])
        
        finalClubData = scores.merge(shooting,how ='left').merge(misc, how='left')
        allClubData.append(finalClubData)
        time.sleep(10)
#     keepList = ['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss', 'Sh', "SoT", 'CrdY', 'CrdR', 'Fls', 'Off']
#     finalClubData.rename(columns=names, inplace=True)
    finalClubData = finalClubData[finalClubData["Competition"] == "Premier League"]
    finalClubData["Team"] = clubName
    result = pd.concat(allClubData, axis=0)
    return result

url = "https://fbref.com/en/comps/9/Premier-League-Stats"
getAllClubData(url)

In [ ]:

# url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# allClubLinks = getClubLinks(url)

# #chose one from all the links
# club = requests.get(allClubLinks[1])
# club_name = allClubLinks[1].split("/")[-1].replace("-Stats","").replace("-", " ")

# #thereisonyl one element
# matches = pd.read_html(club.text, match = "Scores & Fixtures")[0]
# matches.columns
# matches = matches[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]


# shooting = getData("/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
# misc = getData("/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])

# final_data = matches.merge(shooting,how ='left').merge(misc, how='left')
# names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
# final_data.rename(columns=names, inplace=True)
# final_data
# final_data = final_data[final_data["Competition"] == "Premier League"]
# final_data["Team"] = club_name
# final_data

<!-- <a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/keeper/Manchester-City-Match-Logs-All-Competitions">Goalkeeping</a>  /

<a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions">Passing</a>

<a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions">Shooting</a>
 -->